Version 0.0.2.9.2

In [ ]:
import sys
sys.path.append("..")

from include.RandomHelper import check_data_state
check_data_state()

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

# Ereignisse
-------------------------

Im Folgenden soll teilweise abgedeckt durch ein Kolloquium der Ursprung der im Weiteren verwendeten Datensätze näher betrachtet werden $-$ der CMS-Detektor. Es soll der Aufbau des Detektors und die Funktionsweise einzelner Detektorkomponenten besprochen werden. Zudem sollen ebenfalls Ereignisse betrachtet werden, anhand derer wichtige physikalische Größen grafisch dargestellt werden können. Für beide Fälle kann das WebInterface [Ispy-WebGL](https://github.com/cms-outreach/ispy-webgl) verwendet werden.

In [ ]:
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-info">
Identifizieren und benennen Sie alle Zerfälle aus der lokalen Sammlung und halten Sie die Darstellungen einiger davon fest.
</div>

Mit einer Internetverbindung:

In [ ]:
%%html
<iframe src="https://ispy-webgl.web.cern.ch/ispy-webgl/" width="100%" height="700"></iframe>

Ohne eine Internetverbindung: Lokal die `index.html` aus dem [Github Repository](https://github.com/cms-outreach/ispy-webgl) in einem Webbrowser öffnen.

Im Weiteren werden nur Ereignisse aus dem Zerfall in vier Leptonen (Myonen und Elektronen, keine Tau-Leptonen) betrachtet.

# Datenformat
----------------------

Das im Weiteren verwendete Datenformat, aus dem alle notwendigen Größen genommen werden ist ein modifiziertes .CSV Format. Der Vorteil dieses Formates ist, dass immer und zu jeder Zeit die Daten so wie sie auch ein Mensch lesen würde betrachten werden können.

Der Abschnitt dient dazu sich mit dem Datenformat vertraut zu machen um die Aufgabe in dem nachfolgendem Abschnitt durchführen zu können.

Die Trennung einzelner Variablen in einem Event geschieht mithilfe von ";". Die Einträge der einzelnen Leptonen innerhalb eines Events werden dagegen klassisch mit "," getrennt. Damit entsteht der Vorteil einer individuellen Anzahl an Leptonen in einem Event ohne die Einführung von zusätzlichen Platzhaltern.

In [ ]:
%matplotlib inline
#%matplotlib qt
import numpy as np
import pandas as pd

from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

name_1 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_2el2mu_init.csv"
name_2 = "../data/for_long_analysis/mc_init/MC_2012_ZZ_to_4L_4mu_init.csv"

dataframe_1 = pd.read_csv(name_1, delimiter=";")
dataframe_2 = pd.read_csv(name_2, delimiter=";")

In [ ]:
dataframe_1

In [ ]:
dataframe_2

Für die Bearbeitung einzelner Größen können die jeweiligen Elemente aus dem String (`str`) Datenformat wieder zurück in einer Liste (`list`) umgewandelt werden:
Hierzu stehen die Optionen offen entweder die Standardbibliothek `ast`

In [ ]:
import ast
px = ast.literal_eval(f"[{dataframe_2.loc[3, 'px']}]")
px = np.array(px, dtype=float)
px

oder die `split` Methode für die `str` zu vernwenden

In [ ]:
py = dataframe_2.loc[3, 'py'].split(",")
py = np.array(py, dtype=float)
py

Daraus können neue Größen wie der transversale Impuls bestimmt werden anhand dessen eine Eingrenzung in den Größen getroffen werden kann und soll

In [ ]:
pt = np.sqrt(px ** 2 + py ** 2)
pt

Für Myonen muss ein Minimalwert von 5 GeV für den Transversalimpuls überschritten werden. Für Transversalimpuls Werte darunter steigt die Wahrscheinlichkeit einer Missidentifikation der Myonen an.
Insofern müssen alle Myonen in diesem Ereignis, die die Bedingung nicht erfüllen, verworfen werden.

In [ ]:
pt_minimum_filter = pt > 5
pt_minimum_filter

In [ ]:
pt = pt[pt_minimum_filter]
pt

Diese Anwendung des so entstandenen Filters muss damit individuell für ein Ereignis auf jede Variable im Datensatz angewendet werden und das bearbeitete Ereignis wieder abgespeichert werden, was bei einzelner, händischer Anwendung einem viel zu hohem Zeitaufwand entspricht, der aber durch das Einführen einer bereits erstellten Klasse, die das ganze automatisch macht, deutlich reduzieren kann.

# Anwendung der Filter mit Hilfe von Apply
----------------------

Die Anwendung eines Filters erfolgt auf alle Variablen innerhalb eines Ereignisses und ist von dem Filter für das nächste Ereignis verschieden.
Die reihenweise Anwendung des Filters auf den Datensatz ist bereits implementiert und über die Apply Klasse anwendbar.

In [ ]:
from include.processing.Apply import Apply

Bei der Erstellung des Objekts muss zusätzlich eine "allowed" und eine "calculation" Klasse übergeben werden. Die bei der Anwendung der Filter notwendigen Größen oder Filter werden anhand dieser Klassen berechnet.
Für des Beispiel des minimalen Transversalimpulses lassen sich die beiden Klassen als eine Ansammlung von Funktionen darstellen, die alleine stehen könnten (@staticmethod), aber logisch in einer Klasse zusammengefasst werden.

In [ ]:
class Calc_Start(object):
    
    @staticmethod
    def pt(px, py):
        return np.sqrt(px ** 2 + py ** 2)

class Allowed_Start(object):
    
    @staticmethod
    def min_pt(pt, look_for):
        return pt > 7.0 if look_for == "electron" else (
            pt > 5.0 if look_for == "muon" else None)

In [ ]:
print(name_2)
process = Apply(input_=name_2, particle_type="muon", use_n_rows=10000,
                calc_instance=Calc_Start, allowed_instance=Allowed_Start,
               use_swifter=True, multi_cpu=False)

In dem Datensatz `name_2` existiert die Größe des Transversalimpulses noch nicht. In einem Zwischenschritt, der in dem Filter `"check_min_pt"` schon enthalten ist lässt sich der Transversalimpuls auch explizit hinzufügen:

In [ ]:
process.add_variable("pt")
process.data

Die Häufigkeitsverteilung einer solchen Variable kann durch die Methode `hist_of_variable` dargestellt werden.

In [ ]:
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Die Anwendung des Filters für den minimalen Transversalimpuls und die Entfernung aller nachfolgend entstandenen Ereignisse, die weniger als vier Leptonen enthalten liefert die entsprechende Verteilung:

In [ ]:
process.filter(filter_name="check_min_pt")
process.hist_of_variable(variable="pt", bins=100, hist_range=(0, 80))

Ebenso lassen sich auch andere Variablen durch die Häufigkeitsverteilungen visualisieren.
Der durchgeführte Cut beeinflusst hauptsächlich die Verteilung des Transversalimpulses, ändert an der Verteilung anderer Größen dagegen wenig.

<div class="alert alert-info">
Betrachten Sie die Verteilungen einiger in den Datensätzen vorhandenen Größen. Weichen manche der Größen von Ihrer Erwartung ab? Welche wären es und warum.

Sie können die Codefragmente aus diesem Abschnitt ebenfalls für die unteren Aufgaben verwenden.

</div>


In [ ]:
# here goes the code

# Berechnung wichtiger Größen
----------------------

<div class="alert alert-info">

Dem Beispiel des Transversalimpulses folgend implementieren sie alle nachfolgend notwendigen Größen und visualisieren sie angemessen die
  * Pseudorapidität $\eta$
  * Transversalimpuls $p_T$
  * Azimutalwinkel $\phi$

Und erklären sie ihre Beobachtungen.

(Überprüfen Sie ihre Implementierungen mithilfe der MC-Simulationen des Untergrundes.)
</div>

Der dafür verwendete Klassenskelett erbt die vorhergehende Methode für den Transversalimpuls und einer Initialklasse, die Methoden wie die Rekonstruktion der Z-Bosonen Paare enthält.

In [ ]:
class CalcStudent(Calc_Start):
    '''
    Class for the calculation of certain sizes that are used for
    the cuts or are essential for the reconstruction.
    '''

    @staticmethod
    def combined_charge(charge, combine_num):
        '''
        Tests whether an electrically neutral charge combination is possible.

        :param charge: ndarray
                       1D array containing data with "int" type.
        :param combine_num: int
                            4 if look_for is not "both", 2 else
        :return: bool
        '''
        # code

    @staticmethod
    def eta(px=None, py=None, pz=None, energy=None):
        '''
        Calculates the pseudorapidity.
        Optional with or without energy.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                    1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def invariant_mass_square(px, py, pz, energy=None, eta=None, phi=None):
        '''
        Calculates the square of the invariant mass.
        Optional with or without energy.
        Optionally with or without eta and phi.

        :param phi: ndarray
                    1D array containing data with "float" type.
        :param eta: ndarray
                    1D array containing data with "float" type.
        :param px: ndarray
                   1D array containing data with "float" type.
        :param py: ndarray
                   1D array containing data with "float" type.
        :param pz: ndarray
                   1D array containing data with "float" type.
        :param energy: ndarray
                       1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def phi(px, py):
        '''
        Calculation of the angle phi.

        :param px: ndarray
                   1D array containing data with "float" type.
        :param py:  ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def delta_phi(phi1, phi2):
        '''
        Calculation of the difference between two phi angles.

        :param phi1: ndarray
                     1D array containing data with "float" type.
        :param phi2: ndarray
                     1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

    @staticmethod
    def delta_r(eta, phi):
        '''
        Calculation of delta_r.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param phi: ndarray
                    1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "float" type.
        '''
        # code

# Erstellung der Cuts
----------------------

Analog zu dem Cut auf den minimalen Transversalimpuls lassen sich weitere Cuts einführen, die weiter verwendet werden. Ein Beispiel welche Cuts seitens CMS hierfür verwendet wurden kann in der [offiziellen Veröffentlichung](https://arxiv.org/pdf/1207.7235.pdf) nachgelesen werden.

<div class="alert alert-info">

Implementieren sie die in der Klasse 'AllowedStudent' aufgelisteten Methoden. Schätzen sie die Wahl der Cuts mithilfe von den Verteilungen im Kapitel **Anwendung der Filter mithilfe von Apply**, dem **Detektor** und den **kinematischen Einschränkungen** der Ereignisse.

(Überprüfen sie ihre Implementierungen mithilfe der MC-Simulationen des Untergrundes.)
    
</div>

In [ ]:
class AllowedStudent(Allowed_Start):
    '''
    Class that introduces certain cuts and thus restricts the leptons in the events.
    '''
    
    @staticmethod
    def delta_r(delta_r):
        '''
        Checks if delta_r is smaller than the allowed value.

        :param delta_r: ndarray
                        1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code

    @staticmethod
    def rel_pf_iso(rel_pf_iso):
        '''
        Checks if rel_pf_iso is smaller than the allowed value.

        :param rel_pf_iso: ndarray
                           1D array containing data with `float` type.
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code
        
    @staticmethod
    def misshits(misshits):
        '''
        Checks if the minimum number of misshits was kept.

        :param misshits:
        :return:
        '''
        # code
        
    @staticmethod
    def pt(p_t, look_for, coll_size=4):
        '''
        Checks if the exact pedingun regarding pt is observed.
        (>20 GeV: >= 1; >10 GeV: >= 2; >Minimum pt: >= 4).

        :param p_t: ndarray
                    1D array containing data with `float` type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :param coll_size: int
                          4 if look_for is not "both", 2 else
        :return: ndarray
                 1D array containing data with `bool` type.
        '''
        # code
        
    @staticmethod
    def eta(eta, look_for):
        '''
        Checks if the pseudorapidity of leptons is valid.

        :param eta: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def lepton_type(typ, look_for):
        '''
        Checks for the permitted classification of leptons.

        :param typ: ndarray
                    1D array containing data with "float" type.
        :param look_for: str
                         "muon"; "electron" or "both"
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def impact_param(sip3d, dxy, dz):
        '''
        Checks if the impact parameters of the collision are valid and sorts out
        events that do not have a clear and equal collision point.

        :param sip3d: ndarray
                      1D array containing data with "float" type.
        :param dxy: ndarray
                    1D array containing data with "float" type.
        :param dz: ndarray
                   1D array containing data with "float" type.
        :return: ndarray
                 1D array containing data with "bool" type.
        '''
        # code

    @staticmethod
    def zz(z1, z2):
        '''
        Checks if the Z1 candidate and the Z2 candidate is within the allowed range.

        :param z1: float
        :param z2: float
        :return: bool
        '''
        # code

Kombination des implementieren Codes mit den teilweise zur Verfügung gestellten Klassen:

In [ ]:
from include.processing.CalcAndAllowerInit import AllowedInit
from include.processing.CalcAndAllowerInit import CalcInit

AllowedInit.a_allowed_instance = AllowedStudent
AllowedInit.a_calc_instancea = CalcStudent
class Allowed(AllowedStudent, AllowedInit):
    pass
    
CalcInit.c_allowed_instance = Allowed
CalcInit.c_calc_instance = CalcStudent
class Calc(CalcStudent, CalcInit):
    pass

# Anwendung der Filter und Rekonstruktion auf MC - Simulationen
--------------------------------------------------------------------------------------------------------------

Die vorhandenen Implementierungen der Cuts wurden mithilfe der Untergrund-MC-Simulationen getestet. Dies soll das gezielte Hinarbeiten auf ein gewisses Ziel (eigentliche Messung) verhindern, da durch eine geringe Anzahl an späteren Ereignissen in der eigentlichen Messung die Tatsache weitestgehend vermieden werden soll, gezielt Ereignisse in einem Bereich subjektiv auszuwählen.

Sofern die obigen Abschnitte vollständig bearbeitet wurden ist die Aufgabe dieses Abschnittes eine sinnvolle Reihenfolge der oben definierten Funktionen zu finden. Falls die Möglichkeit besteht mehrere CPU Kerne zu verwenden, so wird diese Möglichkeit empfohlen, da diese die Filter und Rekonstruktionszeit entsprechend der Anzahl der CPU Kerne verringern. Nichtsdestotrotz kann auch alles auf einem Kern - mit ein bisschen mehr Zeitaufwand - durchgeführt werden.

Für die nachfolgende Routine ist es sinnvoll sich eine Liste an Tupeln zu erstellen:

In [ ]:
from include.processing.ApplyHelper import ProcessHelper


# Alle Untergrund MC und Signal MC für m = 125 GeV
mc_files = True
# Durchgeführte Messung
run_files = True

# Speicherort der Messung
dir_measurement = "../data/for_long_analysis/ru_init/"
# Speicherort der Untergrund MC und 125 GeV Signal MC
dir_mc = "../data/for_long_analysis/mc_init/"


file_tuples = []
if mc_files:
    file_tuples += ProcessHelper.create_tuple(dir_mc)    

if run_files:
    file_tuples += ProcessHelper.create_tuple(dir_measurement)

Ein einzelner solcher `namedtuple` enthält die Datei und den Teilchentyp des Datensatzes (notwendig für `Apply`):

In [ ]:
file_tuples[0]

Der ganze Filter- und Rekonstruktionsablauf kann in einer Funktion zusammengefasst werden. Die Reihenfolge der Filter ist hierbei für die Laufzeit entscheidend, kann aber auch nicht beliebig geändert werden, da ein logischer Ablauf hinter der Anwendung der Filter und Rekonstruktionen steckt, der nicht beliebig geändert werden kann.

Mit den Zuvor implementierten neuen Größen und den Cuts stehen folgende Filter- und Rekonstruktionsschritte zur Auswahl:

In [ ]:
Apply.help()

Die Funktion kann die folgende Form haben:

In [ ]:
def filter_and_reco_process(used_pair):
    process = Apply(input_=used_pair.name, 
                    particle_type=used_pair.particle, 
                    multi_cpu=True, use_swifter=False,
                    calc_instance=Calc, 
                    allowed_instance=Allowed)
    
    # Logische Reihenfolge Wählen
    # quicksave: Speichert den Datensatz NACH dem Anwenden 
    # des Filter- bzw Rekonstruktionsschrittes
    
    # bereits angewendet um die Reduktion der Datenmengen auf ein Notebook Niveau (1 Kern) zu ermöglichen. 
    # Typ nicht mehr im Datensatz enthalten!
    # process.filter(filter_name="check_type", quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftT"))
    # ProcessHelper.change_on_affix("Name_OldAffix.csv", "NewAffix"):
    # -> Diese Methode Ändert "Name_OldAffix.csv" in "Name_NewAffix.csv"
    #    und speichert "Name_NewAffix.csv" in dem neuen Ordner <ru oder mc>_NewAffix
    
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_min_pt")
    process.filter(filter_name="check_impact_param")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_exact_pt")
    process.filter(filter_name="check_m_2l")
    process.filter(filter_name="check_rel_iso")
    process.filter(filter_name="check_q")
    if process.particle_type != "muon":
        process.filter(filter_name="check_misshit")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_eta")
    process.filter(filter_name="check_q")
    process.filter(filter_name="check_m_4l",
                   quicksave=ProcessHelper.change_on_affix(used_pair.name, "befZ"))
    process.reconstruct(reco_name="zz", 
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftZ"))
    process.reconstruct(reco_name="mass_4l_out_zz",
                        quicksave=ProcessHelper.change_on_affix(used_pair.name, "aftH"))
    del process

<div class="alert alert-info">

Wählen Sie die passende Reihenfolge der Filter- und Rekonstruktionsschritte.

Müssen einige Filterschritte mehrmals durchgeführt werden?und warum ist es nicht sinnvoll die Rekonstruktion der ZZ Bosonen sehr früh durchzuführen?

(Wenn das Interesse besteht eine Zeitoptimierte Version mit den hier vorhandenen Funktionen durchzuführen kann die Geschwindigkeit der einzelnen Filter wie folgt abgeschätzt werden: Es sich den Datensatz der Untergrund-MC-Simulationen zu verwenden und bei der Initialisierung der 'Apply'-Instanz folgende 'kwargs' zu verwenden: 'multi_cpu=False, use_swifter=True'. Nun sollte neben die Geschwindigkeit angezeigt werden, die anzeigt wie viele Zeilen pro Sekunde bearbeitet werden.)
    
</div>

Die Anwendung von `filter_and_reco_process` auf die Datensätze: 

In [ ]:
from tqdm import tqdm
from IPython.display import clear_output

if input("Run all filter + reco (y/n): ") == "y":
    for pair in tqdm(file_tuples):
        filter_and_reco_process(pair)
        clear_output()

# Betrachtung der finalen Verteilungen
------------------------------------------------------------------

Nach der Durchführung des Filter- und Rekonstruktionsschritts können nun die finalen Verteilungen einzelner Größen betrachtet werden. Hierbei wird eine Unterscheidung zwischen der Signal MC, sowie der Untergrund MC gemacht. Die hier verwendeten Signal MC Simulationen sind die eines Higgs-Bosons mit einer Masse von 125 GeV. Die Begründung, warum diese Simulation die passende ist - unter der Berücksichtigung der vorhandenen Messung - wird im zweiten Aufgabenteil durchgeführt, der Ihnen in der Veranstaltung TP2 begegnen wird.

<div class="alert alert-info">

Betrachten Sie die Verteilung der Vier-Leptonen-Invarianten Massen, sowie die Massen der beiden Z-Bosonen.
    
</div>

Des Weiteren sind auch folgende Größen darstellbar (wobei es fraglich ist, ob `z1_index`, `z2_index`, sowie `z1_tag` bzw. `z2_tag` Größen sind die eine ausführlicher Betrachtung bedürfen):

In [ ]:
from include.processing.ApplyHelper import ProcessHelper
print(ProcessHelper.print_possible_variables("../data/for_long_analysis/mc_aftH/MC_2012_H_to_ZZ_to_4L_2el2mu_aftH.csv"))

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)
h = HistOf(mc_dir="../data/for_long_analysis/mc_aftH",
           ru_dir="../data/for_long_analysis/ru_aftH", info=[["2012"], ["B", "C"]])

In [ ]:
# Beispiel
h.variable("energy", 50, (0, 200))
ax = plt.gca()
ax.set_xlabel(r"$p_T$ in GeV")
ax.set_ylabel("Bineinträge")
plt.show()

Die hier dargestellten Unsicherheiten sind asymmetrisch. Hierzu wird von dem Erwartungswert eine Poisson Verteilung (gemessene Anzahl an Ereignissen) die untere Grenze ( - 34%) und die obere Grenze (+34%) bestimmt, womit ein 68% Unsicherheitsintervall angegeben werden kann. Die Asymmetrie der Poissonverteilung ist für einen kleinen Erwartungswert deutlich sichtbar. Erst im Grenzfall großer Erwartungswerte geht die Poissonverteilung in die Gaußverteilung über und die Unsicherheiten auf ein Messwert werden symmetrisch.

In [ ]:
# weitere Verteilungen

# Grobe Abschätzung der statistischen Signifikanz

Die ausführliche Version der Bestimmung der statistischen Signifikanz findet im zweiten Teil statt. Eine grobe Abschätzung kann aber durch $$ Z \approx \frac{s}{\sqrt{b}} $$ durchgeführt werden, wobei $b$ die Anzahl der erwarteten Untergrundereignisse ist, die der MC Simulation entnommen wird. Für das Signal $s$ wird die Differenz zwischen der gesamten Anzahl der Messungen und dem erwartetem Untergrund (wieder aus der MC Simulation) entnommen.


<div class="alert alert-info">

Schätzen Sie mit der oben aufgeführten Formel die Signifikanz des Higgs Bosons mit der Masse von 125 GeV ab. Welche Aussagen können Sie über den Wert machen? Welche Probleme hat diese Art von Abschätzung und warum ist sie nicht wirklich für eine quantitative Aussage geeignet?
    
</div>


In [ ]:
# usefull code
_, hist = h.variable("mass_4l", 15, (100, 150))
mc_sig, mc_sig_bac, measurement = hist.data["mc_sig"], histogramm.data["mc_bac"], histogramm.data["data"]

In [ ]:
# your code